In [1]:
import json
with open(file="../../echo-chamber/data/mastodon.trump.json") as f:
    data = json.load(f)

In [3]:
import re
from bs4 import BeautifulSoup
md = []
for post in data:
    if not post['in_reply_to_id']:
        content = post["content"]
        # Parse the HTML content with BeautifulSoup
        soup = BeautifulSoup(content, "html.parser")
    
        # Remove all 'a' tags entirely
        for a_tag in soup.find_all("a"):
            a_tag.unwrap()
    
        # Get the plain text
        plain_text = soup.get_text(separator=" ")
    
        # Regular expression to match and remove URLs
        cleaned_content = re.sub(
            r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F])|\s)+",
            "",
            plain_text,
        )
    
        # Remove excess spaces left by removing the link
        cleaned_content = re.sub(r"\s+", " ", cleaned_content).strip()
        md.append([post['id'],cleaned_content])

In [ ]:
import pandas as pd
md_df = pd.DataFrame(md, columns=['id','post'])

In [3]:
import json
with open(file="../../echo-chamber/data/bsky.trump.json") as f:
    bsky = json.load(f)


In [4]:
bsky_data = []
from fast_langdetect import detect_language
for post in bsky:
    if not post['record'].get('reply',None):
        if post['record'].get('text',None):
            language = post['record'].get('langs',None)
            language = language[0] if language else None
            if not language:
                language = detect_language(post['record']['text'].replace('\n',' ')).lower()
            bsky_data.append([post['_id'],post['record']['text'], language])

In [5]:
import pandas as pd
bsky_df = pd.DataFrame(bsky_data, columns=['id','post', 'lang'])

In [6]:
bsky_df['lang'].value_counts()[:10]

In [7]:
bsky_df['post'].nunique()

In [1]:
import json
with open(file="../../echo-chamber/data/truthsocial.trump.json") as f:
    ts = json.load(f)


In [2]:
import re
from bs4 import BeautifulSoup
ts_data = []
for post in ts:
    if post['in_reply_to_id'] or post['quote_id']:
        continue
    else:
        content = post["content"]
        # Parse the HTML content with BeautifulSoup
        soup = BeautifulSoup(content, "html.parser")
    
        # Remove all 'a' tags entirely
        for a_tag in soup.find_all("a"):
            a_tag.unwrap()
    
        # Get the plain text
        plain_text = soup.get_text(separator=" ")
    
        # Regular expression to match and remove URLs
        cleaned_content = re.sub(
            r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F])|\s)+",
            "",
            plain_text,
        )
    
        # Remove excess spaces left by removing the link
        cleaned_content = re.sub(r"\s+", " ", cleaned_content).strip()
        if cleaned_content:
            ts_data.append([post['id'],cleaned_content])

In [10]:
ts_data[:10]


In [11]:
ts_df = pd.DataFrame(ts_data, columns=['id','post'])

In [12]:
ts_df['id'].nunique()

In [ ]:
sum(bsky_df['post'] == '')

In [9]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from sentence_transformers import SentenceTransformer, util
import numpy as np

# Step 1: Initialize the SBERT model
model = SentenceTransformer('all-MiniLM-L6-v2', )  # You can use other SBERT models as well

# Step 2: Define your two lists
list1 = list(bsky_df['post'])
list2 = list(ts_df['post'])

# Step 3: Compute embeddings for both lists
embeddings1 = model.encode(list1, convert_to_tensor=True, show_progress_bar=True, batch_size=512)
embeddings2 = model.encode(list2, convert_to_tensor=True, show_progress_bar=True, batch_size=512)



In [ ]:

# Step 4: Compute cosine similarity
cosine_sim = util.cos_sim(embeddings1.cpu(), embeddings2.cpu())

In [ ]:
similarity_matrix = cosine_sim
# Set threshold for similarity
threshold = 0.75

# Filter indices where similarity > threshold
row, col = np.where(similarity_matrix > threshold)
weights = similarity_matrix[row, col]



In [ ]:
from scipy.sparse import coo_matrix
import networkx as nx
# Use sparse representation for performance
sparse_sim = coo_matrix((weights, (row, col)), shape=similarity_matrix.shape)

# Initialize bipartite graph
B = nx.Graph()

# Add nodes for Platform A (0 to len(embeddings_a) - 1) and Platform B
B.add_nodes_from(range(len(embeddings1)), bipartite=0)
B.add_nodes_from(range(len(embeddings1), len(embeddings1) + len(embeddings2)), bipartite=1)

# Add edges from sparse similarity data
edges = [(r, len(embeddings1) + c, {"weight": w}) for r, c, w in zip(sparse_sim.row, sparse_sim.col, sparse_sim.data)]
B.add_edges_from(edges)

# Print graph information
print(f"Bipartite graph has {B.number_of_nodes()} nodes and {B.number_of_edges()} edges.")

In [ ]:
components = list(nx.connected_components(B))

In [ ]:
#find component larger than 1
components = [c for c in components if len(c) > 1]


In [ ]:
print(f"Number of components larger than 1: {sum(np.array(num_components) > 1)}")

In [ ]:
# Example: Retrieve posts for each component
platform_a_posts = list1
platform_b_posts = list2
component_posts = []
for component in components:
    posts = {
        "Platform A": [platform_a_posts[node] for node in component if node < len(platform_a_posts)],
        "Platform B": [platform_b_posts[node - len(platform_a_posts)] for node in component if node >= len(platform_a_posts)],
    }
    component_posts.append(posts)

In [ ]:
component_posts

In [19]:
len(all_posts)

In [39]:
bsky_posts = list(bsky_df.loc[(bsky_df['lang'] == 'en'),'post'].dropna())

#drop post contain "event":"initializing","ts
bsky_posts = [post for post in bsky_posts if "event\":\"initializing\",\"ts\"" not in post]
bsky_posts = [post for post in bsky_posts if detect_language(post.replace('\n','')).lower() == 'en' ]
# ts_df posts remove digits and hyperlinks
ts_posts = list(ts_df['post'].dropna())


all_posts = bsky_posts + ts_posts
target_posts = [post for post in all_posts if "trump" in post.lower() or "biden" in post.lower()]

In [13]:

from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from nltk.corpus import stopwords
import nltk
import cudf
from sentence_transformers import SentenceTransformer
from cuml.manifold import UMAP
from cuml.cluster import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.dimensionality import BaseDimensionalityReduction
from sklearn.cluster import KMeans

cluster_model = KMeans(n_clusters=20)
# Fit BERTopic without actually performing any dimensionality reduction
empty_dimensionality_model = BaseDimensionalityReduction()

# # Initialize multilingual stopwords
# languages = ['EN', 'JA', 'DE', 'FR', 'PT', 'IT', 'ZH']

# # Define custom stopword lists for each language
# stopword_lists = {
#     'EN': stopwords.words('english'),
#     'DE': stopwords.words('german'),
#     'FR': stopwords.words('french'),
#     'PT': stopwords.words('portuguese'),
#     'IT': stopwords.words('italian'),
#     # For unsupported languages, provide your own stopword lists
#     'JA': ['これ', 'それ', 'あれ', 'この', 'その', 'あの', 'ここ', 'そこ', 'あそこ', 'こちら', 'どれ', 'なぜ', 'なに', 'どうして'],  # Example
#     'ZH': ['的', '一', '是', '在', '了', '和', '有', '不', '人', '我', '他', '这', '个', '上', '们', '来'],  # Example
# }
# # Combine and deduplicate stopwords
# all_stopwords = set()
# for lang, stopword_list in stopword_lists.items():
#     all_stopwords.update(stopword_list)

# all_stopwords = list(all_stopwords)  # Convert back to a list if needed
all_stopwords = stopwords.words('english')
custom_stopwords = ['com', 'www', '2024', 'http', 'https', "bsky", "social"]  # Add domain-specific stopwords
all_stopwords.extend(custom_stopwords)
all_stopwords = list(set(all_stopwords))  # Deduplicate again


# # Initialize the CountVectorizer with the custom stopwords
vectorizer_model = CountVectorizer(stop_words=all_stopwords, min_df=2, ngram_range=(1, 2))

embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

umap_model = UMAP(n_components=15, n_neighbors=5, min_dist=0.1)
hdbscan_model = HDBSCAN(min_cluster_size=200, min_samples=10, metric='euclidean', cluster_selection_method='eom', prediction_data=True)
# Combine posts from each component
# bsky_df posts remove digits and hyperlinks and only keep en
bsky_posts = list(bsky_df.loc[(bsky_df['lang'] == 'en'),'post'].dropna())

#drop post contain "event":"initializing","ts
bsky_posts = [post for post in bsky_posts if "event\":\"initializing\",\"ts\"" not in post]
bsky_posts = [post for post in bsky_posts if detect_language(post.replace('\n','')).lower() == 'en' ]
# ts_df posts remove digits and hyperlinks
ts_posts = list(ts_df['post'].dropna())


all_posts = bsky_posts + ts_posts
target_posts = [post for post in all_posts if "trump" in post.lower() or "biden" in post.lower()]
# Fit BERTopic model
embeddings = embedding_model.encode(target_posts, show_progress_bar=True)
topic_model = BERTopic(
    umap_model=empty_dimensionality_model,
    hdbscan_model=hdbscan_model,
    embedding_model=embedding_model,
    vectorizer_model=vectorizer_model,
    verbose=True,
)
topics, probs = topic_model.fit_transform(target_posts, embeddings)




In [14]:
topic_df = topic_model.get_topic_info()

In [16]:
new_topics = topic_model.reduce_outliers(target_posts, topics)

In [15]:
topic_df

In [17]:
topic_model.update_topics(target_posts, topics=new_topics, vectorizer_model=vectorizer_model)


In [19]:
topic_df = topic_model.get_topic_info()
topic_df

In [33]:
topic_df.to_csv("../data/topic_df.csv", index=False)

In [59]:
#label topics
# Topic -1: Criticism of Trump and Support for Democratic Policies
# Topic 0: MAGA and Pro-Trump Hashtags and Advocacy
# Topic 1: Trump’s Legal Convictions and Felony Charges
# Topic 2: Pro-Trump and MAGA Advocacy
# Topic 3: Celebrations of Trump (e.g., Birthdays and Tributes)
# Topic 4: Hunter Biden’s Legal Troubles (e.g., Gun Charges)
# Topic 5: U.S. Policy on Ukraine and Russia
# Topic 6: Israel-Hamas Conflict and Biden’s Ceasefire Proposal
# Topic 7: Trump’s Tax Promises and Election Campaign
# Topic 8: Trump’s Rallies and Live Events Coverage
# Topic 9: Biden’s Immigration Policies and Executive Orders
# Topic 10: Legal Proceedings in Georgia’s 2020 Election Case Against Trump
# Topic 11: Biden vs. Trump Presidential Debates
topic_id_to_label = {-1: "Criticism of Trump and Support for Democratic Policies",
                                0: "MAGA and Pro-Trump Hashtags and Advocacy",
                                1: "Trump’s Legal Convictions and Felony Charges",
                                2: "Pro-Trump and MAGA Advocacy",
                                3: "Celebrations of Trump (e.g., Birthdays and Tributes)",
                                4: "Hunter Biden’s Legal Troubles (e.g., Gun Charges)",
                                5: "U.S. Policy on Ukraine and Russia",
                                6: "Israel-Hamas Conflict and Biden’s Ceasefire Proposal",
                                7: "Trump’s Tax Promises and Election Campaign",
                                8: "Trump’s Rallies and Live Events Coverage",
                                9: "Biden’s Immigration Policies and Executive Orders",
                                10: "Legal Proceedings in Georgia’s 2020 Election Case Against Trump",
                                11: "Biden vs. Trump Presidential Debates"}
topic_model.set_topic_labels({-1: "Criticism of Trump and Support for Democratic Policies",
                                0: "MAGA and Pro-Trump Hashtags and Advocacy",
                                1: "Trump’s Legal Convictions and Felony Charges",
                                2: "Pro-Trump and MAGA Advocacy",
                                3: "Celebrations of Trump (e.g., Birthdays and Tributes)",
                                4: "Hunter Biden’s Legal Troubles (e.g., Gun Charges)",
                                5: "U.S. Policy on Ukraine and Russia",
                                6: "Israel-Hamas Conflict and Biden’s Ceasefire Proposal",
                                7: "Trump’s Tax Promises and Election Campaign",
                                8: "Trump’s Rallies and Live Events Coverage",
                                9: "Biden’s Immigration Policies and Executive Orders",
                                10: "Legal Proceedings in Georgia’s 2020 Election Case Against Trump",
                                11: "Biden vs. Trump Presidential Debates"})
topic_model.get_topic_info().to_csv("../data/custom_topic_df.csv", index=False)

In [20]:
all_post_topics_df = pd.DataFrame({"post": target_posts, "topic": new_topics})

In [20]:
#save models
embedding_model = "sentence-transformers/all-MiniLM-L6-v2"
topic_model.save("../models/", serialization="safetensors", save_ctfidf=True, save_embedding_model=embedding_model)


In [1]:
#load models
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
topic_model = BERTopic.load("../models/")
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")


In [27]:
topic_model.visualize_topics()

In [2]:

topic_model.visualize_hierarchy(custom_labels=True)

In [ ]:
# Reduce dimensionality of embeddings, this step is optional but much faster to perform iteratively:
reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)
# Visualize the documents in 2-dimensional space and show the titles on hover instead of the abstracts
# NOTE: You can hide the hover with `hide_document_hover=True` which is especially helpful if you have a large dataset
# NOTE: You can also hide the annotations with `hide_annotations=True` which is helpful to see the larger structure
topic_model.visualize_documents(titles, reduced_embeddings=reduced_embeddings, custom_labels=True)


In [62]:
posts_in_null = list(all_post_topics_df.loc[all_post_topics_df['topic'] == -1, 'post'])
# run topic model on posts in null
topic_model_null = BERTopic(
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    embedding_model=embedding_model,
    vectorizer_model=vectorizer_model,
    verbose=True,
)
topics_null, probs_null = topic_model_null.fit_transform(posts_in_null)

In [63]:
topic_df_null = topic_model_null.get_topic_info()

In [56]:

import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from gensim.corpora.dictionary import Dictionary
import nltk

# Download NLTK resources
#nltk.download('punkt_tab')
#nltk.download('stopwords')
#nltk.download('wordnet')


# Preprocessing function
def preprocess_text(text, language='english'):
    # Initialize lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    # Lowercase the text
    text = text.lower()
    
    # Remove URLs, mentions, hashtags, and special characters
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)  # URLs
    text = re.sub(r"@\w+|#\w+", "", text)  # Mentions and hashtags
    text = re.sub(r"[^\w\s]", "", text)  # Special characters
    
    # Remove emojis
    text = re.sub(r"[^\x00-\x7F]+", "", text)  # Non-ASCII characters (e.g., emojis)
    
    # Tokenize the text
    tokens = word_tokenize(text)
    
    # Remove stopwords and short tokens
    stop_words = set(stopwords.words(language))
    stop_words.update(set(['com', 'www', 'http', 'https', "bsky", "social", "bskysocial",'link','permalink', 'trump','biden']))  # Custom stopwords
    tokens = [word for word in tokens if word not in stop_words and len(word) > 2]
    
    # Lemmatize tokens
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Return processed tokens
    return tokens

documents = target_posts 
# Preprocess documents
preprocessed_docs = [preprocess_text(doc) for doc in documents]

# Create a dictionary and a corpus
dictionary = Dictionary(preprocessed_docs)
corpus = [dictionary.doc2bow(doc) for doc in preprocessed_docs]
#using LDA
from gensim.models import LdaModel
from gensim.models.coherencemodel import CoherenceModel

# Define a function to compute coherence scores for different topic numbers
def compute_coherence_values(corpus, dictionary, texts, start=2, limit=10, step=1):
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = LdaModel(
            corpus=corpus,
            id2word=dictionary,
            num_topics=num_topics,
            random_state=42,
            passes=20,
            alpha='auto',
            per_word_topics=True,
        )
        model_list.append(model)
        coherence_model = CoherenceModel(
            model=model, texts=texts, dictionary=dictionary, coherence='c_v'
        )
        coherence_values.append(coherence_model.get_coherence())
    return model_list, coherence_values

# Compute coherence scores
start, limit, step = 2, 10, 1
models, coherence_values = compute_coherence_values(
    corpus=corpus, dictionary=dictionary, texts=preprocessed_docs, start=start, limit=limit, step=step
)

# Print coherence values
for num_topics, coherence in zip(range(start, limit, step), coherence_values):
    print(f"Num Topics = {num_topics}, Coherence Score = {coherence}")




In [57]:
import matplotlib.pyplot as plt
import seaborn as sns
x = range(start, limit, step)
sns.lineplot(x=x, y=coherence_values, marker='o', markerfacecolor='white', markeredgecolor='blue', linestyle='-', )
plt.xlabel("Number of Topics")
plt.ylabel("Coherence Score")
plt.title("Coherence Score vs Number of Topics")
plt.show()


In [58]:
#print the 4 topics result
for idx, model in enumerate(models):
    if idx == 1:
        print(f"Number of topics: {idx + 2}")
        topics = model.print_topics(num_words=10)
        for topic in topics:
            print(topic)
        print("\n")

In [59]:
#label the posts with lda topics
model = models[1]
lda_topics = [model.get_document_topics(corpus[i]) for i in range(len(corpus))]
lda_topics = [max(topic, key=lambda x: x[1])[0] for topic in lda_topics]
# Create a DataFrame with the topics
lda_df = pd.DataFrame({"post": documents, "topic": lda_topics})



In [22]:
# assign platfomr to the post
# using fast way
ts_post_set = set(ts_posts)
bertopic_df = all_post_topics_df.copy() 
bertopic_df['platform']  = all_post_topics_df['post'].apply(lambda x: 'ts' if x in ts_post_set else 'bsky')

In [63]:
ts_df_nonull = ts_df.dropna(subset=['post']).reset_index(drop=True)
ts_df_nonull = ts_df_nonull.merge(bertopic_df, on='post', how='left')

In [64]:
ts_df_nonull.drop_duplicates(subset=['id'], inplace=True)

In [65]:
ts_df_nonull.dropna(subset=['topic'], inplace=True)

In [66]:
ts_df_nonull['topic'] = ts_df_nonull['topic'].astype(int)
ts_df_nonull['topic'] = ts_df_nonull['topic'].map(topic_id_to_label)
ts_df_nonull.to_csv("../data/ts_df_id_topic.csv", index=False)

In [62]:
#bsky df
bsky_df_nonull = bsky_df.dropna(subset=['post']).reset_index(drop=True)
bsky_df_nonull = bsky_df_nonull.merge(bertopic_df, on='post', how='left')
bsky_df_nonull.drop_duplicates(subset=['id'], inplace=True)
bsky_df_nonull.dropna(subset=['topic'], inplace=True)
bsky_df_nonull['topic_label'] = bsky_df_nonull['topic'].apply(lambda x: topic_id_to_label[int(x)])
bsky_df_nonull.to_csv("../data/bsky_df_id_topic.csv", index=False)

In [61]:
bsky_df_nonull

In [1]:
ts_df_nonull.topic.value_counts()

In [ ]:
data = []
# Iterate over components and map posts to their topics and IDs
for component in component_topics:
    # Process Platform A (bsky)
    for post, topic in zip(component["Component"]["Platform A"], component["Topics"][:len(component["Component"]["Platform A"])]):
        match_bsky = bsky_df[bsky_df["post"] == post]
        if not match_bsky.empty:
            data.append({"id": match_bsky["id"].values[0], "post": post, "topic": topic})
    
    # Process Platform B (ts)
    for post, topic in zip(component["Component"]["Platform B"], component["Topics"][len(component["Component"]["Platform A"]):]):
        match_ts = ts_df[ts_df["post"] == post]
        if not match_ts.empty:
            data.append({"id": match_ts["id"].values[0], "post": post, "topic": topic})


In [ ]:
import pandas as pd


# Collect all posts and their associated topics and component IDs
all_data = []
for component_id, component in enumerate(component_topics):  # Use enumerate to assign unique component_id
    # Add Platform A (bsky) posts
    all_data.extend([(component_id, post, topic) for post, topic in zip(
        component["Component"]["Platform A"], 
        component["Topics"][:len(component["Component"]["Platform A"])]
    )])
    
    # Add Platform B (ts) posts
    all_data.extend([(component_id, post, topic) for post, topic in zip(
        component["Component"]["Platform B"], 
        component["Topics"][len(component["Component"]["Platform A"]):]
    )])# Create a DataFrame for all posts with their component_id and topic
all_posts_df = pd.DataFrame(all_data, columns=["component_id", "post", "topic"])

# Merge with bsky_df
bsky_merged = all_posts_df.merge(bsky_df, on="post", how="inner")

# Merge with ts_df
ts_merged = all_posts_df.merge(ts_df, on="post", how="inner")

# Combine the results
export_df = pd.concat([bsky_merged, ts_merged])

# Reorder columns and export to CSV
export_df = export_df[["component_id", "id", "post", "topic"]]

## Analysis based on control


In [2]:
import pandas as pd
bsky_topic_df = pd.read_csv("../data/bsky_df_id_topic.csv")
ts_topic_df = pd.read_csv("../data/ts_df_id_topic.csv")

In [4]:
#Bluesky thread api
import requests
import json
import os

def get_thread(thread_id):
    url = f"https://public.api.bsky.app/xrpc/app.bsky.feed.getPostThread"
    params = {
        "uri":thread_id,
        'depth': 1000
    }
    response = requests.get(url, params=params)
    return response.json()
def get_repost(post_id):
    url = f"https://public.api.bsky.app/xrpc/app.bsky.feed.getRepostedBy"
    params = {
        "uri":post_id,
        "limit": 100
    }
    response = requests.get(url, params=params)
    return response

In [ ]:
response = get_repost(gaza_ids[61])

In [ ]:
#get respse headers
response.json()

In [ ]:
export_df['id']

In [ ]:
get_thread(export_df['id'][12395])['thread']['replies']

In [ ]:
get_repost("at://did:plc:m73xlht5hftrkos34nojlchj/app.bsky.feed.post/3lbk3kho7ak2n").json()


In [7]:
bsky_threads = []
from tqdm.auto import tqdm
for bsky_id in tqdm(bsky_topic_df['id']):
    if "at" not in bsky_id:
        break
    thread = get_thread(bsky_id)
    bsky_threads.append(thread) 

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
# Function to recursively traverse and add edges to the graph
def build_network(graph, thread, parent_id=None):
    post = thread['post']
    post_id = post['uri']  # Use the URI as a unique identifier
    graph.add_node(post_id, text=post['record']['text'], author=post['author']['displayName'])
    
    if parent_id:  # If there is a parent, add an edge
        graph.add_edge(parent_id, post_id)
    
    # Process replies recursively
    for reply in thread.get('replies', []):
        build_network(graph, reply, post_id)

# Create a directed graph
G = nx.DiGraph()

# Build the graph
for thread in bsky_threads:
    try:
        build_network(G, thread['thread'])
    except:
        print(thread)


In [ ]:
components = [G.subgraph(c).copy() for c in nx.weakly_connected_components(G)]
    

In [76]:
import networkx as nx
from collections import defaultdict, deque

# Function to calculate width and depth of a tree
def calculate_width_and_depth(tree):
    # Choose an arbitrary root (assuming it's a directed graph)
    roots = [node for node in tree.nodes if tree.in_degree(node) == 0]
    if not roots:
        return 0, 0  # Empty tree case

    root = roots[0]  # Choose the first root
    levels = defaultdict(int)
    max_depth = 0

    # Perform BFS to calculate levels
    queue = deque([(root, 0)])  # (node, level)
    visited = set()

    while queue:
        node, depth = queue.popleft()
        if node in visited:
            continue
        visited.add(node)
        levels[depth] += 1
        max_depth = max(max_depth, depth)
        for neighbor in tree.neighbors(node):
            queue.append((neighbor, depth + 1))

    max_width = max(levels.values())  # Width is the maximum nodes at any level
    return max_width, max_depth

# Function to analyze all trees in a forest
def analyze_forest(graph):
    # For directed graphs, use weakly connected components to identify trees
    components = [graph.subgraph(c).copy() for c in nx.weakly_connected_components(graph)]
    results = []
    for i, tree in enumerate(components, 1):
        #give me root id
        width, depth = calculate_width_and_depth(tree)
        results.append({"Tree_root_id": 
                        [node for node in tree.nodes if tree.in_degree(node) == 0],
                        "Tree_id": i, "Width": width, "Depth": depth})
    return results

#resutls = analyze_forest(G)

In [ ]:
resutls

In [ ]:
len(components)

In [ ]:
len(bsky_threads)


In [ ]:
gaza_ids = set(gaza_ids)
gaza_bsky = []
for post in bsky:
    if post['_id'] in gaza_ids:
        gaza_bsky.append(post)
        

In [ ]:
gaza_bsky_posts = []
for post in gaza_bsky:
    gaza_bsky_posts.append([post["_id"], post['record']['text'],post['record']['createdAt'], post['replyCount'], post['repostCount'],])


In [ ]:
gaza_bsky_df = pd.DataFrame(gaza_bsky_posts, columns=['id','post','createdAt','replyCount','repostCount'])

In [ ]:
gaza_bsky_df['replyCount'].hist()

In [5]:
gaza_ts = []
from tqdm.auto import tqdm
for post in tqdm(ts):
    if post["_id"] in ts_topic_df['id']:
        gaza_ts.append(post)


In [ ]:
gaza_ts_posts = []
for post in gaza_ts:
    gaza_ts_posts.append([post["_id"], post['content'],post['created_at'], post['replies_count'], post['reblogs_count'], post['favourites_count']])


In [ ]:
gaza_ts_df = pd.DataFrame(gaza_ts_posts, columns=['id','post','createdAt','replyCount','repostCount','favouriteCount'])

In [ ]:
gaza_ts_df['replyCount']

In [40]:
export_df.loc[export_df['component_id'] == 0, 'id']

In [ ]:

# Create the DataFrame
export_df = pd.DataFrame(data)


In [ ]:
export_df.to_csv("../data/matched_bsky_ts.csv", index=False)

In [ ]:
# Save BERTopic model
topic_model.save("../models/bertopic_model")

# Export topic summary
topic_info = topic_model.get_topic_info()
topic_info.to_csv("../data/bertopic_topic_summary.csv", index=False)

# Extract and save top words for each topic
topics_dict = {}
for topic_id in topic_info["Topic"]:
    if topic_id != -1:  # Skip the "outlier" topic (-1)
        topics_dict[topic_id] = topic_model.get_topic(topic_id)

# Save topic keywords to a CSV
topics_keywords = []
for topic_id, keywords in topics_dict.items():
    keywords_str = ", ".join([f"{word} ({score:.2f})" for word, score in keywords])
    topics_keywords.append({"Topic": topic_id, "Keywords": keywords_str})

topics_df = pd.DataFrame(topics_keywords)
topics_df.to_csv("../data/bertopic_keywords.csv", index=False)


In [ ]:
export_df

In [4]:
import networkx as nx
from datetime import datetime
from collections import defaultdict
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
def generate_network(nodes):
    """
    Generate a network graph from a list of nodes with 'id', 'in_reply_to_id', and metadata.

    :param nodes: List of dictionaries containing:
                  - 'id': Unique identifier for the post
                  - 'in_reply_to_id': ID of the parent post this replies to
                  - Metadata such as username, like counts, etc.
    :return: A NetworkX DiGraph with nodes and edges.
    """
    G = nx.DiGraph()  # Directed graph to represent reply relationships

    for node in tqdm(nodes):
        # Add the current node with metadata
        G.add_node(
            int(node['id']),
            # username=node['account']['username'],
            # display_name=node['account']['display_name'],
            # verified=node['account']['verified'],
            # followers_count=node['account']['followers_count'],
            # statuses_count=node['account']['statuses_count'],
            # replies_count=node['replies_count'],
            # favourites_count=node['favourites_count'],
            # reblogs_count=node['reblogs_count'],
            # visibility=node['visibility'],
            # content=node['content'],
            # created_at=node['created_at'],
            # language=node['language'],
            # sensitive=node['sensitive'],
            # mentions=[mention['username'] for mention in node.get('mentions', [])],
        )

        # If it replies to another node, add an edge
        if node['in_reply_to_id']:
            G.add_node(int(node['in_reply_to_id']))  # Add the parent node if it doesn't exist
            G.add_edge(int(node['id']), int(node['in_reply_to_id']))

    return G

# Generate the network graph
ts_network = generate_network(ts)

  0%|          | 0/3556587 [00:00<?, ?it/s]

In [5]:
#find the root of the trees
ts_trees = nx.weakly_connected_components(ts_network)
ts_ids = set(ts_topic_df['id'])
# filter the tress which the root is in the trump_biden
trump_biden_ts_trees = []
root_ids = []
for tree in tqdm(ts_trees):
    #define root node of the tree: no in degree
    root = [node for node in tree if ts_network.out_degree(node) == 0]
    if (len(root) == 1) and (root[0] in ts_ids):
        trump_biden_ts_trees.append(tree)
        root_ids.append(root[0])
    elif len(root) >= 2:
        print(root)

len(trump_biden_ts_trees)

0it [00:00, ?it/s]

43818

In [6]:
#using nx combine all the trees
ts_combined = nx.compose_all([ts_network.subgraph(tree).copy() for tree in trump_biden_ts_trees])

In [10]:
ts_nodes = list(ts_combined.nodes())

In [13]:
filtered_ts = []
ts_nodes = set(ts_nodes)
for post in tqdm(ts):
    if int(post['_id']) in ts_nodes:
        filtered_ts.append(post)

  0%|          | 0/3556587 [00:00<?, ?it/s]

In [15]:
with open(file="../data/ts_threads.json", mode="w") as f:
    json.dump(filtered_ts, f)

In [77]:
ts_results = analyze_forest(ts_combined)

In [68]:
#pirnt tree which width larger than 20
for tree in ts_results:
    if tree['Width'] > 20:
        print(tree)

In [39]:
#write ts_gaza to json
import json
with open("../data/topic_corpus/bsky_gaza.json", "w") as f:
    json.dump(bsky_threads, f)


In [53]:
analyze_forest(bsky_trump_biden)

In [42]:
#fetch components 0
from tqdm.auto import tqdm
trump_biden = export_df.loc[export_df['component_id'] == 0, 'id']

bsky_trump_biden = []
for bsky_id in tqdm(trump_biden):
    if "at" not in bsky_id:
        break
    thread = get_thread(bsky_id)
    bsky_trump_biden.append(thread) 

In [45]:
with open("../data/topic_corpus/bsky_trump_biden.json", "w") as f:
    json.dump(bsky_trump_biden, f)

In [70]:
import networkx as nx
import matplotlib.pyplot as plt
# Function to recursively traverse and add edges to the graph
def build_network(graph, thread, parent_id=None):
    post = thread['post']
    post_id = post['uri']  # Use the URI as a unique identifier
    graph.add_node(post_id, text=post['record']['text'], author=post['author']['displayName'])
    
    if parent_id:  # If there is a parent, add an edge
        graph.add_edge(parent_id, post_id)
    
    # Process replies recursively
    for reply in thread.get('replies', []):
        build_network(graph, reply, post_id)

# Create a directed graph
bsky_trump_biden_nw = nx.DiGraph()
error_count = 0
# Build the graph
for thread in tqdm(bsky_trump_biden):
    try:
        build_network(bsky_trump_biden_nw, thread['thread'])
    except:
        error_count += 1

In [78]:
bsky_results = analyze_forest(bsky_trump_biden_nw)

In [79]:
for tree in bsky_results:
    if tree['Width'] > 20 or tree['Depth'] > 10:
        print(tree)

In [46]:
# Example dataset
all_descendants = []
# Example usage
for root_id in tqdm(trump_biden):
    if "at" in root_id:
        continue
    descendants = find_descendants(ts, root_id)
    all_descendants.extend(descendants)

In [80]:
import networkx as nx
import matplotlib.pyplot as plt

def analyze_tree_structure(ts_forest, bsky_forest):
    """
    Compare tree structure metrics (depth, width, size) between two forests.
    
    :param ts_forest: NetworkX DiGraph for TruthSocial.
    :param bsky_forest: NetworkX DiGraph for Bluesky.
    """
    def calculate_metrics(forest):
        depths, widths, sizes = [], [], []
        for component in nx.weakly_connected_components(forest):
            tree = forest.subgraph(component)
            root = [node for node in tree.nodes if tree.in_degree(node) == 0][0]
            
            # Depth
            depth = nx.dag_longest_path_length(tree)
            depths.append(depth)
            
            # Width
            level_counts = {}
            for node in tree.nodes:
                level = len(nx.ancestors(tree, node))
                level_counts[level] = level_counts.get(level, 0) + 1
            widths.append(max(level_counts.values(), default=0))
            
            # Size
            sizes.append(len(tree.nodes))
        
        return depths, widths, sizes
    
    ts_depths, ts_widths, ts_sizes = calculate_metrics(ts_forest)
    bsky_depths, bsky_widths, bsky_sizes = calculate_metrics(bsky_forest)
    
    # Plot results
    plt.figure(figsize=(12, 6))
    plt.boxplot([ts_depths, bsky_depths], labels=["TruthSocial", "Bluesky"])
    plt.title("Tree Depth Comparison")
    plt.ylabel("Depth")
    plt.show()

    plt.figure(figsize=(12, 6))
    plt.boxplot([ts_widths, bsky_widths], labels=["TruthSocial", "Bluesky"])
    plt.title("Tree Width Comparison")
    plt.ylabel("Width")
    plt.show()

    plt.figure(figsize=(12, 6))
    plt.boxplot([ts_sizes, bsky_sizes], labels=["TruthSocial", "Bluesky"])
    plt.title("Tree Size Comparison")
    plt.ylabel("Number of Nodes")
    plt.show()


In [83]:
analyze_tree_structure(ts_network, bsky_trump_biden_nw)

In [87]:
import networkx as nx
import pandas as pd

# Example: Forest as a single directed graph
# Edges define relationships between nodes in different trees

# Function to compute metrics for a single tree
def compute_tree_metrics(tree):
    tree_depth = nx.dag_longest_path_length(tree) if nx.is_directed_acyclic_graph(tree) else 0
    branching_factor = (
        sum(dict(tree.out_degree()).values()) / tree.number_of_nodes()
        if tree.number_of_nodes() > 0 else 0
    )
    root_nodes = [node for node in tree.nodes() if tree.in_degree(node) == 0]
    root_activity = len(list(tree.successors(root_nodes[0]))) if root_nodes else 0
    leaf_nodes = [node for node in tree.nodes() if tree.out_degree(node) == 0]
    num_leaf_nodes = len(leaf_nodes)
    
    return {
        "tree_depth": tree_depth,
        "branching_factor": branching_factor,
        "root_activity": root_activity,
        "num_leaf_nodes": num_leaf_nodes,
    }

# Analyze each tree (connected component) in the forest
metrics = []
for component in nx.weakly_connected_components(bsky_trump_biden_nw):
    subgraph = bsky_trump_biden_nw.subgraph(component)  # Extract subgraph for the tree
    metrics.append(compute_tree_metrics(subgraph))

# Convert metrics to a DataFrame
metrics_df = pd.DataFrame(metrics)

# Summarize the metrics across the forest
summary = metrics_df.describe()

# Display the summary
print("Summary of Forest Metrics:")
print(summary)


In [88]:
metrics = []
for component in nx.weakly_connected_components(ts_combined):
    subgraph = ts_combined.subgraph(component)  # Extract subgraph for the tree
    metrics.append(compute_tree_metrics(subgraph))

# Convert metrics to a DataFrame
metrics_df = pd.DataFrame(metrics)

# Summarize the metrics across the forest
summary = metrics_df.describe()

# Display the summary
print("Summary of Forest Metrics:")
print(summary)


In [102]:
len(set(trump_biden))

In [129]:
#using igraph to analyze the motifs
import igraph as ig
# Create an iGraph Graph object from the NetworkX graph
ig_graph = ig.Graph(directed=False)
ig_graph.add_vertices(list(ts_combined.nodes))
ig_graph.add_edges(list(ts_combined.edges))

# Find motifs in the graph
motifs_ts = ig_graph.motifs_randesu(size=4)

# Display the motifs
motifs_ts

In [128]:
from automotif import AutoMotif
from dotmotif import executors
import networkx as nx
#convert bsky_trump_biden_nw to undirected graph
bsky_trump_biden_nw_undirected = nx.Graph(bsky_trump_biden_nw)
# Set up AutoMotif for GPU accelerated motif finding
motif_finder = AutoMotif(Graph=bsky_trump_biden_nw_undirected, size=4, directed=False, verbose=True, use_GPU=True)

# Start finding motifs
motifs = motif_finder.find_all_motifs()

# # Calculate the Z-Score for the motifs
# z_scores = motif_finder.calculate_zscore(num_random_graphs = 30, Executor = executors.NetworkXExecutor)

# Display the motifs found
motif_finder.display_all_motifs()

In [121]:
#for bluesky
# Create an iGraph Graph object from the NetworkX graph
ig_graph = ig.Graph(directed=False)
ig_graph.add_vertices(list(bsky_trump_biden_nw.nodes))
ig_graph.add_edges(list(bsky_trump_biden_nw.edges))

# Find motifs in the graph
motifs_bsky = ig_graph.motifs_randesu(size=4)

# Display the motifs
motifs_bsky

In [122]:
from igraph import Graph, plot
from pprint import pprint
# Number of vertices
n = 4

# Iterate over all possible isomorphism classes for 4-vertex directed graphs
# The number of isomorphism classes for directed graphs with 4 vertices is 218
for class_id in range(16):
    # Generate the graph for the given isomorphism class
    motif_graph = Graph.Isoclass(n=n, cls=class_id, directed=False)
    
    # Display the graph
    pprint(f"Isomorphism Class ID: {class_id}: {motif_graph}")
    


In [108]:
ig.Graph.Isoclass(n=3, class=1)

In [148]:
import networkx as nx


# Define the motif graph (e.g., a star)
motif = nx.Graph()
motif.add_edges_from([(0, 1), (1, 2), (2, 3), (3,4)])
motif_star = nx.Graph()
motif_star.add_edges_from([(0, 1), (0, 2), (0, 3), (0,4)])
#break down to each tree and detect the motif
# Iterate over each tree in the forest
count = 0
count_star = 0
for component in nx.connected_components(bsky_trump_biden_nw_undirected):
    subgraph = bsky_trump_biden_nw_undirected.subgraph(component)  # Extract subgraph for the tree
    # Find all subgraphs isomorphic to the motif
    motif_count = 0
    matcher = nx.algorithms.isomorphism.GraphMatcher(subgraph, motif)
    for match in matcher.subgraph_isomorphisms_iter():
        motif_count += 1
        count += 1
    matcher_star = nx.algorithms.isomorphism.GraphMatcher(subgraph, motif_star)
    for match in matcher_star.subgraph_isomorphisms_iter():
        count_star += 1



In [149]:
count_star

In [150]:
count

# fetch the reblog authors

In [1]:
import json
with open("../data/bsky_reposts.json", "r") as f:
    bsky_threads = json.load(f)

In [34]:
from tqdm.auto import tqdm
all_authors = set()
for i in tqdm(bsky_threads):
    all_authors.add(i['author']['did'])
    if i.get('reposts',None) and i['reposts'] != []:
        for j in i['reposts']:
            all_authors.add(j['did'])


  0%|          | 0/195616 [00:00<?, ?it/s]

In [37]:
if None:
    print("None")

In [5]:
import requests

def fetch_flower_information(base_url, author, limit=100):
    """
    Fetches flower information from the API using pagination.

    Parameters:
        base_url (str): The API endpoint.
        author (str): The author ID.
        limit (int): Number of items to fetch per request.

    Returns:
        list: A list of all flower information retrieved from the API.
    """
    all_flower_info = []
    params = {
        "actor": author,
        "limit": limit
    }

    while True:
        response = requests.get(base_url, params=params)

        if response.status_code != 200:
            print(f"Error: Received status code {response.status_code}")
            break

        data = response.json()
        
        # Add the flower information from the current response
        all_flower_info.extend(data.get("follows", []))
        #all_flower_info.extend(data.get("followers", []))

        # Check if there's a cursor for the next page
        cursor = data.get("cursor")
        if not cursor:
            break

        # Update the params with the new cursor
        params["cursor"] = cursor

    return all_flower_info


In [6]:
    author = "did:plc:f6vzq5ful3a3x2mq4hc5qkvi"
    url = "https://public.api.bsky.app/xrpc/app.bsky.graph.getFollows"
    follows = fetch_flower_information(url, author)

In [8]:
follows

[{'did': 'did:plc:4zu6i4ri34lgbfpzz3a2walg',
  'handle': 'survivingtrump.bsky.social',
  'displayName': 'SurvivingTrump',
  'avatar': 'https://cdn.bsky.app/img/avatar/plain/did:plc:4zu6i4ri34lgbfpzz3a2walg/bafkreiegzw5cu2a43l3mmt5wrppqp57fatkgpjmssrg4tkzllm4qr76bu4@jpeg',
  'labels': [],
  'createdAt': '2024-11-14T14:09:22.926Z',
  'description': 'They’ve robbed my Peace of Mind! Staying informed, embracing political activism, stepping out of my comfort zone.  \nWharton MBA, Eagles and Phillies fan, pickleball, golf, duplicate bridge (distracted now by politics) \nColorado gal now',
  'indexedAt': '2024-12-22T23:53:09.243Z'},
 {'did': 'did:plc:gxi5cs4kimxwjbbinxniznwo',
  'handle': 'catalinashelby.bsky.social',
  'displayName': 'StrongerTogether',
  'avatar': 'https://cdn.bsky.app/img/avatar/plain/did:plc:gxi5cs4kimxwjbbinxniznwo/bafkreigx44ehtrfpz52iqjideorduecbrbkd46zeypqvu35ggrfcx25pym@jpeg',
  'labels': [],
  'createdAt': '2024-11-11T13:16:23.189Z',
  'description': 'Doing my part 

In [9]:
import requests
import json
def fetch_flower_information(base_url, author, limit=100):
    """
    Fetches flower information from the API using pagination.

    Parameters:
        base_url (str): The API endpoint.
        author (str): The author ID.
        limit (int): Number of items to fetch per request.

    Returns:
        list: A list of all flower information retrieved from the API.
    """
    all_flower_info = []
    params = {
        "actor": author,
        "limit": limit
    }

    while True:
        response = requests.get(base_url, params=params)

        if response.status_code != 200:
            print(f"Error: Received status code {response.status_code}")
            break

        data = response.json()
        
        # Add the flower information from the current response
        all_flower_info.extend(data.get("follows", []))
        #all_flower_info.extend(data.get("followers", []))

        # Check if there's a cursor for the next page
        cursor = data.get("cursor")
        if not cursor:
            break

        # Update the params with the new cursor
        params["cursor"] = cursor

    return all_flower_info

if __name__ == "__main__":
    base_url = "https://public.api.bsky.app/xrpc/app.bsky.graph.getFollows"
    from tqdm.auto import tqdm
    with open("../data/bsky_reposts.json", "r") as f:
        bsky_threads = json.load(f)
    all_authors = set()
    for i in tqdm(bsky_threads[:2]):
        all_authors.add(i['author']['did'])
        if i.get('reposts',None) and i['reposts'] != []:
            for j in i['reposts']:
                all_authors.add(j['did'])
    print(f"Total authors: {len(all_authors)}")
    authors = list(all_authors)
    all_flower_info = []
    for author in tqdm(authors):
        all_flower_info.append({author: fetch_flower_information(base_url, author)})


  0%|          | 0/2 [00:00<?, ?it/s]

Total authors: 68


  0%|          | 0/68 [00:00<?, ?it/s]

Error: Received status code 400


KeyboardInterrupt: 

In [10]:
all_flower_info